In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix
from sklearn.decomposition import TruncatedSVD
from IPython.display import display

# Load the data
ratings_df = pd.read_csv("Preprocessed_Ratings.csv")
books_df = pd.read_csv("Preprocessed_Books.csv")

# Map Book_Rating to implicit feedback (1 for positive feedback, 0.5 for neutral, 0 for negative)
ratings_df['implicit_feedback'] = ratings_df['Book_Rating'].apply(lambda x: 1 if x >= 7 else (0.5 if x >= 4 else 0))

# Step 1: Map Ratings to Review Sentiment (Proxy)
def map_rating_to_review(rating):
    if rating >= 7:
        return 'Positive'
    elif rating >= 4:
        return 'Neutral'
    else:
        return 'Negative'

ratings_df['Review_Sentiment'] = ratings_df['Book_Rating'].apply(map_rating_to_review)

# Limit the dataset to top 10 users and books
top_users = ratings_df['User_ID'].value_counts().head(10).index
top_books = ratings_df['ISBN'].value_counts().head(10).index
ratings_df = ratings_df[ratings_df['User_ID'].isin(top_users) & ratings_df['ISBN'].isin(top_books)]

# Check for missing values
print(ratings_df.isnull().sum())

# Create the sparse matrix for User-Book interaction
rows = ratings_df['User_ID'].astype('category').cat.codes
cols = ratings_df['ISBN'].astype('category').cat.codes
data = ratings_df['implicit_feedback']

interaction_matrix = coo_matrix((data, (rows, cols)), shape=(ratings_df['User_ID'].nunique(), ratings_df['ISBN'].nunique()))

# Perform Matrix Factorization using SVD
n_components = min(interaction_matrix.shape[1], 50)  # Ensure n_components <= number of books
svd = TruncatedSVD(n_components=n_components)
matrix_factorized = svd.fit_transform(interaction_matrix)

# Reconstruct the interaction matrix
reconstructed_matrix = np.dot(matrix_factorized, svd.components_)

# Step 2: Recommend books for a specific user
def recommend_books_for_user(user_id, top_n=5):
    """
    Recommend books for a specific user based on predicted interaction scores.
    
    Args:
        user_id (int): The user ID for whom recommendations are needed.
        top_n (int): Number of books to recommend.
    
    Returns:
        pd.DataFrame: Recommended books with book details and user reviews.
    """
    if user_id not in ratings_df['User_ID'].unique():
        raise ValueError(f"User ID {user_id} not found in the dataset.")
    
    # Get the predicted interaction scores for the user
    user_index = ratings_df['User_ID'].unique().tolist().index(user_id)
    user_predictions = reconstructed_matrix[user_index]
    
    # Get books the user has already interacted with
    interacted_books = ratings_df[ratings_df['User_ID'] == user_id]['ISBN'].tolist()
    
    # Filter out books the user has already interacted with
    recommended_books = pd.Series(user_predictions, index=ratings_df['ISBN'].unique())
    recommended_books = recommended_books[~recommended_books.index.isin(interacted_books)]
    
    # Get top N book ids based on predicted interaction scores
    top_book_ids = recommended_books.sort_values(ascending=False).head(top_n).index.tolist()
    
    # Fetch book details from the Books dataframe
    recommended_books_details = books_df[books_df['ISBN'].isin(top_book_ids)].copy()
    
    # Step 3: Add review sentiments (based on ratings) to the recommended books
    reviews_df = ratings_df[ratings_df['ISBN'].isin(top_book_ids)]
    reviews_df = reviews_df.groupby('ISBN').agg({'Book_Rating': 'mean', 'Review_Sentiment': 'first'}).reset_index()

    # Merge book details with review sentiments
    recommended_books_details = pd.merge(recommended_books_details, reviews_df, on='ISBN', how='left')
    
    return recommended_books_details[['ISBN', 'Book_Title', 'Book_Author', 'Book_Rating', 'Review_Sentiment']]

# Step 4: Recommend books for all users
all_recommended_books = []

# Iterate over all unique users in the ratings data
for user_id in ratings_df['User_ID'].unique():
    top_books = recommend_books_for_user(user_id, top_n=5)
    top_books['User_ID'] = user_id  # Add user_id to the recommendation dataframe
    all_recommended_books.append(top_books)

# Combine all the recommendations into a single dataframe
all_recommended_books_df = pd.concat(all_recommended_books, ignore_index=True)

# Display the results in a table format
display(all_recommended_books_df)

Rating_ID            0
User_ID              0
ISBN                 0
Book_Rating          0
implicit_feedback    0
Review_Sentiment     0
dtype: int64


,ISBN,Book_Title,Book_Author,Book_Rating,Review_Sentiment,User_ID
0,0060928336,Divine Secrets of the Ya-Ya Sisterhood: A Novel,Rebecca Wells,0.0,Negative,246655
1,0312195516,The Red Tent (Bestselling Backlist),Anita Diamant,0.0,Negative,246655
2,0316666343,The Lovely Bones: A Novel,Alice Sebold,5.0,Positive,246655
3,0446672211,Where the Heart Is (Oprah's Book Club (Paperba...,Billie Letts,0.0,Negative,246655
4,059035342X,Harry Potter and the Sorcerer's Stone (Harry P...,J. K. Rowling,3.4,Positive,246655
5,0142001740,The Secret Life of Bees,Sue Monk Kidd,0.0,Negative,254465
6,0385504209,The Da Vinci Code,Dan Brown,3.0,Negative,254465
7,044023722X,A Painted House,John Grisham,0.0,Negative,254465
8,059035342X,Harry Potter and the Sorcerer's Stone (Harry P...,J. K. Rowling,3.4,Positive,254465
9,067976402X,Snow Falling on Cedars,David Guterson,8.0,Positive,254465


### popularity based filtering

In [3]:
import pandas as pd
from IPython.display import display  # Import display function

# Load preprocessed data
books_df = pd.read_csv("Preprocessed_Books.csv")
ratings_df = pd.read_csv("Preprocessed_Ratings.csv")

# Standardize column names
books_df.rename(columns={'Book-Title': 'Book_Title'}, inplace=True)
ratings_df.rename(columns={'User-ID': 'User_ID', 'Book-Rating': 'Book_Rating'}, inplace=True)

# Calculate book popularity based on the number of ratings and average rating
book_popularity = ratings_df.groupby('ISBN').agg(
    num_ratings=('Book_Rating', 'count'),
    avg_rating=('Book_Rating', 'mean')
).reset_index()

# Merge with book details
book_popularity = book_popularity.merge(books_df, on='ISBN', how='inner')

# Define popularity score (weighted sum of avg rating and number of ratings)
book_popularity['popularity_score'] = book_popularity['num_ratings'] * book_popularity['avg_rating']

# Get top 10 popular books
top_books = book_popularity.sort_values(by='popularity_score', ascending=False).head(10)

# Get the top 10 users based on the number of ratings
top_users = ratings_df['User_ID'].value_counts().head(10).index

# Recommend popular books to the top 10 users
recommendations = {user: top_books['Book_Title'].tolist() for user in top_users}

# Display recommendations using display
for user, rec_books in recommendations.items():
    display(f"\nRecommendations for User {user}:")
    for book in rec_books:
        display(f" - {book}")

'\nRecommendations for User 269566:'

' - The Lovely Bones: A Novel'

' - The Da Vinci Code'

' - The Red Tent (Bestselling Backlist)'

" - Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))"

...

" - The Pilot's Wife : A Novel"

' - Wild Animus'

' - Divine Secrets of the Ya-Ya Sisterhood: A Novel'

' - Life of Pi'

' - The Nanny Diaries: A Novel'

In [4]:
import pandas as pd

# Load preprocessed data
books_df = pd.read_csv("Preprocessed_Books.csv")
ratings_df = pd.read_csv("Preprocessed_Ratings.csv")

# Standardize column names
books_df.rename(columns={'Book-Title': 'Book_Title'}, inplace=True)
ratings_df.rename(columns={'User-ID': 'User_ID', 'Book-Rating': 'Book_Rating'}, inplace=True)

# Calculate book popularity based on the number of ratings and average rating
book_popularity = ratings_df.groupby('ISBN').agg(
    num_ratings=('Book_Rating', 'count'),
    avg_rating=('Book_Rating', 'mean')
).reset_index()

# Merge with book details
book_popularity = book_popularity.merge(books_df, on='ISBN', how='inner')

# Define popularity score (weighted sum of avg rating and number of ratings)
book_popularity['popularity_score'] = book_popularity['num_ratings'] * book_popularity['avg_rating']

# Get top 10 popular books
top_books = book_popularity.sort_values(by='popularity_score', ascending=False).head(10)

# Get the top 10 users based on the number of ratings
top_users = ratings_df['User_ID'].value_counts().head(10).index

# Recommend popular books to the top 10 users
recommendations = {user: top_books['Book_Title'].tolist() for user in top_users}

# Display recommendations
for user, rec_books in recommendations.items():
    print(f"\nRecommendations for User {user}:")
    for book in rec_books:
        print(f" - {book}")


Recommendations for User 269566:
 - The Lovely Bones: A Novel
 - The Da Vinci Code
 - The Red Tent (Bestselling Backlist)
 - Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))
 - Where the Heart Is (Oprah's Book Club (Paperback))
 - The Pilot's Wife : A Novel
 - Wild Animus
 - Divine Secrets of the Ya-Ya Sisterhood: A Novel
 - Life of Pi
 - The Nanny Diaries: A Novel

Recommendations for User 257204:
 - The Lovely Bones: A Novel
 - The Da Vinci Code
 - The Red Tent (Bestselling Backlist)
 - Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))
 - Where the Heart Is (Oprah's Book Club (Paperback))
 - The Pilot's Wife : A Novel
 - Wild Animus
 - Divine Secrets of the Ya-Ya Sisterhood: A Novel
 - Life of Pi
 - The Nanny Diaries: A Novel

Recommendations for User 275970:
 - The Lovely Bones: A Novel
 - The Da Vinci Code
 - The Red Tent (Bestselling Backlist)
 - Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))
 - Where the Heart Is (Oprah's Book Clu

###  recommending books based on readers mood¶


In [6]:
import pandas as pd

# Load preprocessed book data
books_df = pd.read_csv("Preprocessed_Books.csv")

# Categorize books based on mood
mood_book_mapping = {
    "Happy": [],
    "Sad": [],
    "Energetic": [],
    "Tired": [],
    "Moody": [],
    "Relaxed": []
}

# Distribute books from books_df into mood categories
for index, row in books_df.iterrows():
    mood = index % len(mood_book_mapping)  # Assign books cyclically to moods
    mood_name = list(mood_book_mapping.keys())[mood]
    mood_book_mapping[mood_name].append(row['Book_Title'])

# Function to get mood from user
def get_user_mood():
    questions = [
        "How are you feeling right now? (Happy, Sad, Energetic, Tired, Moody, Relaxed)",
        "Do you want something inspiring or reflective?",
        "Are you looking for something light or deep?",
        "Do you prefer fiction or non-fiction?",
        "Would you like a fast-paced or slow-paced book?",
        "Do you enjoy thought-provoking stories or easy-going narratives?",
        "Are you in the mood for a classic or a modern book?",
        "Would you prefer a short read or a long immersive experience?"
    ]
    
    print("Answer the following questions to get a book recommendation:")
    mood = input(questions[0]).strip().capitalize()
    
    if mood in mood_book_mapping:
        return mood
    else:
        print("Invalid mood. Defaulting to Relaxed.")
        return "Relaxed"

# Generate book recommendation based on mood
user_mood = get_user_mood()
recommended_books = mood_book_mapping[user_mood]

# Display recommendations
print(f"\nRecommended books for your mood ({user_mood}):")
for book in recommended_books:
    print(f" - {book}")

Answer the following questions to get a book recommendation:


KeyboardInterrupt: Interrupted by user

In [ ]:
import pandas as pd
import random

# Load books dataset (you need your books CSV for this part)
books_df = pd.read_csv('Preprocessed_Books.csv')

# Define questions to ask the reader with predefined options
questions_with_options = {
    "How do you feel today?": ["Cheerful", "Moody", "Calm", "Tired"],
    "What do you want to do right now?": ["Laugh and have fun", "Reflect and relax", "Go on an adventure", "Read something emotional"],
    "How would you describe your energy level?": ["High energy", "Low energy", "Balanced energy", "Very low energy"],
    "What kind of emotions do you feel today?": ["Happy, excited", "Sad, nostalgic", "Peaceful, content", "Motivated, adventurous"],
    "If you could escape to a different world, what would you choose?": ["A world full of fun and excitement", "A calm, tranquil place", "A fast-paced, action-filled world", "A romantic, dream-like place"]
}

# Define mood categories based on answers (for simplicity, we use a random answer mapping)
mood_categories = {
    "happy": ["Cheerful", "Happy, excited", "Laugh and have fun", "High energy"],
    "sad": ["Moody", "Sad, nostalgic", "Low energy", "Read something emotional"],
    "relaxed": ["Calm", "Reflect and relax", "Balanced energy", "A calm, tranquil place"],
    "energetic": ["Motivated, adventurous", "Go on an adventure", "High energy", "A fast-paced, action-filled world"],
    "romantic": ["Tired", "Read something emotional", "Very low energy", "A romantic, dream-like place"]
}

# Function to ask questions and guess the mood
def guess_mood():
    print("Please answer the following questions to determine your mood:")
    
    answers = []
    for question, options in questions_with_options.items():
        print(f"{question}")
        for i, option in enumerate(options, 1):
            print(f"{i}. {option}")
        answer = int(input(f"Please select an option (1-{len(options)}): "))
        answers.append(options[answer - 1])
    
    # Simple approach: randomly assign a mood based on answers
    mood_guess = random.choice(list(mood_categories.keys()))
    
    print(f"Based on your answers, your mood is predicted as: {mood_guess}")
    return mood_guess

# Map mood to book recommendations (now using keywords instead of specific titles)
mood_to_keywords = {
    "happy": ["joy", "laugh", "fun", "adventure", "feel-good"],
    "sad": ["sad", "nostalgia", "tears", "heartbreak", "grief"],
    "relaxed": ["calm", "peaceful", "reflect", "relax", "serene"],
    "energetic": ["action", "thrill", "mystery", "adventure", "exciting"],
    "romantic": ["love", "romance", "dream", "relationship", "heart"]
}

# Function to recommend books based on mood
def recommend_books_based_on_mood(mood_guess, top_n=5):
    # Map mood to keywords
    recommended_keywords = mood_to_keywords.get(mood_guess, [])
    
    # Filter books by title based on keywords
    filtered_books = books_df[books_df['Book_Title'].str.contains('|'.join(recommended_keywords), case=False, na=False)]
    
    # If no books match the keywords, recommend top N books from the dataset
    if filtered_books.empty:
        filtered_books = books_df.head(top_n)
    
    return filtered_books[['Book_Title', 'Book_Author']].head(top_n)

# Run the mood prediction and book recommendation
reader_mood = guess_mood()
recommended_books = recommend_books_based_on_mood(reader_mood)

# Display the recommended books
print("\nHere are some book recommendations for you:")
print(recommended_books)